# Formaty plików, brakujące wartości

Julian Zubek, 2015, [DELab UW](http://www.delab.uw.edu.pl/)

## Zapis i odczyt plików

Biblioteka pandas oferuje funkcje do odczytu i zapisu kilku popularnych formatów danych. Zajmiemy się tutaj dwoma z nich: prostym formatem tekstowym CSV (*comma separated values*) oraz arkuszami Excela. Na początek przyjrzymy się funkcji .read_csv

In [1]:
def print_file(path):
    f = open(path, "r")
    print(f.read())
    f.close()
    
print_file("../dane/sample1.csv")

Imię,Nazwisko,Wiek
Jan,Kowalski,31
Anna,Kowal,38



In [2]:
import pandas as pd

pd.read_csv("../dane/sample1.csv")

,Imię,Nazwisko,Wiek
0,Jan,Kowalski,31
1,Anna,Kowal,38


W podstawowym wariancie funkcja .read_csv oczekuje pliku tekstowego, w którym pierwsza linia zawiera nazwy kolumn a wartości rozdzielone są przecinkami. Można zmodyfikować domyślne zachowanie podając dodatkowe parametry:

In [3]:
print_file("../dane/sample2.csv")

Jan Kowalski 31
Anna Kowal 38



In [4]:
#sep -- znak separujący wartości
#header -- numer wiersza zawierającego nagłówek lub None w przypadku braku nagłówka
pd.read_csv("../dane/sample2.csv", sep=" ", header=None)

,0,1,2
0,Jan,Kowalski,31
1,Anna,Kowal,38


W niektórych sytuacjach możemy chcieć użyć istniejącej kolumny danych jako indeksu rekordów.

In [5]:
print_file("../dane/sample3.csv")

Imię,Nazwisko,PESEL,Wiek
Jan,Kowalski,84072516931,31
Anna,Kowal,77080317124,38



In [6]:
#index_col -- numer kolumny zawierającej indeks rekordów
pd.read_csv("../dane/sample3.csv", index_col=2)

,Imię,Nazwisko,Wiek
PESEL,,,
84072516931,Jan,Kowalski,31
77080317124,Anna,Kowal,38


Analogicznie możemy wczytać dokument Excela (uwaga: może wymagać doinstalowania pakietu xlrd):

In [7]:
pd.read_excel("../dane/sample3.xlsx", index_col=2)

,Imię,Nazwisko,Wiek
PESEL,,,
84072516931,Jan,Kowalski,31
77080317124,Anna,Kowal,38


W przypadku dokumentu z wieloma arkuszami domyślnie zostanie wczytany pierwszy z nich. Możemy wybrać konkretny arkusz dodając parametr:

In [8]:
pd.read_excel("../dane/sample3.xlsx", sheetname="Arkusz2")

,Nr telefonu,Należność
0,6522345,10
1,4325633,20


Do zapisu danych służą metody .to_csv oraz .to_excel:

In [9]:
dane = pd.read_csv("../dane/sample3.csv", index_col=2)
dane.loc[59060917274] = ("Otello", "Nijakiewicz", 56)
dane

,Imię,Nazwisko,Wiek
PESEL,,,
84072516931,Jan,Kowalski,31
77080317124,Anna,Kowal,38
59060917274,Otello,Nijakiewicz,56


In [10]:
dane.to_csv("../dane/test.csv")
print_file("../dane/test.csv")

PESEL,Imię,Nazwisko,Wiek
84072516931,Jan,Kowalski,31
77080317124,Anna,Kowal,38
59060917274,Otello,Nijakiewicz,56



In [11]:
dane.to_csv("../dane/test.csv", sep=" ", index=False)
print_file("../dane/test.csv")

Imię Nazwisko Wiek
Jan Kowalski 31
Anna Kowal 38
Otello Nijakiewicz 56



In [12]:
dane.to_excel("../dane/test.xlsx")

## Brakujące wartości

Rzeczywistość nie jest idealna i często opiera się pełnemu opisowi – operując na rzeczywistych danych często spotykamy się z brakującymi wartościami. Tabele pandas również pozwalają na ich obsługę w naturalny sposób.

In [33]:
#na_values -- lista symboli oznaczających brak danych
dane = pd.read_csv("../dane/sample4.csv", na_values=["?"]) 
dane

,Imię,Nazwisko,PESEL,Wiek
0,Jan,Kowalski,84072516931,31
1,Anna,Kowal,77080317124,38
2,Otello,Nijakiewicz,NaN,NaN


Symbol NaN oznacza brakującą wartość. Możemy sprawdzić, czy wartość istnieje przy pomocy funkcji isnull:

In [34]:
pd.isnull(dane.loc[2, "Wiek"])

True

In [35]:
pd.isnull(dane.loc[1, "Wiek"])

False

In [36]:
dane.isnull()

,Imię,Nazwisko,PESEL,Wiek
0,False,False,False,False
1,False,False,False,False
2,False,False,True,True


Puste wartości pozostają neutralne względem większości operacji. Można wykonywać na nich działania arytmetyczne, ale wynikiem jest zawsze pusta wartość.

In [37]:
dane["Wiek"] += 2
dane

,Imię,Nazwisko,PESEL,Wiek
0,Jan,Kowalski,84072516931,33
1,Anna,Kowal,77080317124,40
2,Otello,Nijakiewicz,NaN,NaN


W końcu, możliwe jest automatyczne wypełnienie brakujących wartości przez wartość wybraną przez nas:

In [38]:
dane.fillna(0)

,Imię,Nazwisko,PESEL,Wiek
0,Jan,Kowalski,84072516931,33
1,Anna,Kowal,77080317124,40
2,Otello,Nijakiewicz,0,0


In [40]:
dane["Wiek"] = dane["Wiek"].fillna(30)
dane

,Imię,Nazwisko,PESEL,Wiek
0,Jan,Kowalski,84072516931,33
1,Anna,Kowal,77080317124,40
2,Otello,Nijakiewicz,NaN,30
